## 1.数据处理

In [1]:
import pandas as pd
import json
from bs4 import BeautifulSoup

In [2]:
df_06 = pd.read_csv("./202201-06.csv", encoding="gbk", encoding_errors="ignore")
df_09 = pd.read_csv("./202207-09.csv", encoding="gbk", encoding_errors="ignore")
df_12 = pd.read_csv("./202210-12.csv", encoding="gbk", encoding_errors="ignore")
df_02 = pd.read_csv("./202301-02.csv", encoding="gbk", encoding_errors="ignore")
df_05 = pd.read_csv("./202303-05.csv", encoding="gbk", encoding_errors="ignore")
df_08 = pd.read_csv("./202306-08.csv", encoding="gbk", encoding_errors="ignore")

In [3]:
df = pd.concat([df_06, df_09, df_12, df_02, df_05, df_08], ignore_index=True)

In [4]:
df_shape = df.shape
df_shape

(49010, 6)

In [5]:
df

,docartname,doc_art_content,ssubmitdemandcode,reqName,reqType,status
0,解约协议文件导入,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5
1,导入结果查询,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5
2,存量协议文件导入,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5
3,非联网签约,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5
4,?后管配置上线荣数SIP权益服务内容,"<div class=""odm-word"" data-docid=""690696198726...",YWXQ2021120438,权益管理系统接入荣数SIP需求报告,1.0,5
...,...,...,...,...,...,...
49005,贴现利率优惠券的领取和查询,"<div class=""odm-word"" data-docid=""709968605443...",CP0050323060525,票据宝贴现利率优惠券需求,1.0,"3,192"
49006,拆出资金(行社之间)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192"
49007,拆出资金(境内境外)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192"
49008,拆入资金(境内境外)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192"


### 1.1.创建需求状态字典

In [6]:
df_status = pd.read_excel("./需求状态枚举.xlsx")

In [7]:
key_column = df_status["状态码"]
value_column = df_status["状态名称"]
status_dict = dict(zip(key_column, value_column))


{2: '需求管理岗确认',
 5: '需求已上线',
 6: '受理',
 7: '项目技术审批',
 8: '撤销',
 3109: '开发分管领导审批',
 3120: '驳回',
 3190: '业务确认',
 3191: '技术确认',
 3192: '需求开发',
 3193: '测试',
 3209: '架构审批',
 3242: '待上线',
 3269: '需求补充',
 3508: '缺陷已上线',
 3509: '缺陷已关闭',
 3548: '审批流程审查',
 3577: '基线申请',
 3579: '缺陷修复',
 3580: '需求协调',
 3581: '测试确认',
 3625: '基线确认',
 3918: '反洗钱材料审批',
 3919: '需求关闭',
 3925: '反洗钱材料上传',
 3949: '需求规格材料上传',
 3950: '需求规格评审',
 4156: '挂起',
 4723: '项目业务审批',
 4724: '业务前置审批',
 4725: '内部审批',
 4846: '行领导审批'}

### 1.2.数据转换

In [24]:
def format_text(item):
    # 1.handle div element.
    # text_all = ""
    # soup = BeautifulSoup(item)    
    # div_elements = soup.find_all("div")
    # if not div_elements:
    #     return text_all
    # for div in div_elements:
    #     div_text = div.text
    #     # div_text = div_text.replace(" ", "").replace("\n", "").replace("\t", "")
    #     div_text = div_text.replace(" ", "").replace("\t", "")
    #     if not div_text:
    #         continue
    #     text_all += div_text
    soup = BeautifulSoup(item) 
    text_all = soup.get_text()
    text_all = text_all.replace(" ", "").replace("\t", "")
    if not text_all:
        return ""
        
    # 2.handle h1 element.
    h1_element = soup.find("h1")
    h1_text = h1_element.text
    h1_text = h1_text.replace(" ", "")
    
    # 3. concat h1 and div text with "|||".
    if text_all.startswith(h1_text):
        text_all = text_all[len(h1_text):]
    # text_all = h1_text + "|||" + text_all

    return text_all

In [25]:
def format_dataframe(df: pd.DataFrame, column_name: str) -> pd.DataFrame:
    df[f"{column_name}_formated"] = df[column_name].apply(lambda x: format_text(x))
    # df["len"] = df[f"{column_name}_formated"].apply(lambda x: len(x))
    return df

In [26]:
df_formated = format_dataframe(df=df, column_name="doc_art_content")

In [27]:
df_formated

,docartname,doc_art_content,ssubmitdemandcode,reqName,reqType,status,doc_art_content_formated
0,解约协议文件导入,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n非联网产品接收渠道解约协议文件导入请求，逐条调用签约组件进行解约。\n界面原型...
1,导入结果查询,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n非联网产品接收渠道结果查询请求，返回渠道结果文件。\n界面原型*\n中间业务属...
2,存量协议文件导入,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n非联网产品接收渠道存量协议导入请求，并逐条调用签约组件进行签约。\n界面原型*...
3,非联网签约,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n1 非联网产品接收渠道签约报文，调用签约组件进行签约。\n界...
4,?后管配置上线荣数SIP权益服务内容,"<div class=""odm-word"" data-docid=""690696198726...",YWXQ2021120438,权益管理系统接入荣数SIP需求报告,1.0,5,服务产品创建功能描述*\n1、 后管需要配置荣数SIP相关的权益服务内容。\n2、 对于服务...
...,...,...,...,...,...,...,...
49005,贴现利率优惠券的领取和查询,"<div class=""odm-word"" data-docid=""709968605443...",CP0050323060525,票据宝贴现利率优惠券需求,1.0,"3,192",功能描述\n在企业互联pc端领取和查询优惠券。\n输入输出说明\n菜单新增位置：企业互联-票...
49006,拆出资金(行社之间)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192",功能描述*\n优化拆出资金（行社之间，包括本级）的SWIFT行社清算记账规则，取消核心手工记...
49007,拆出资金(境内境外)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192",功能描述*\n优化拆出资金（境内境外）的SWIFT行社清算记账规则，取消核心手工记账步骤，使...
49008,拆入资金(境内境外)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192",功能描述*\n优化拆入资金（境内境外）的SWIFT行社清算记账规则，取消核心手工记账步骤，使...


In [28]:
df_formated_shape = df_formated.shape[0]
df_formated_shape

49010

In [29]:
df_formated["status_cn"] = df_formated["status"].apply(lambda x: status_dict[int(x.replace(",", ""))])

In [30]:
type_dict = {
    "1.0": "产品研发需求",
    "2.0": "数据库表需求",
    "3.0": "特色系统需求",
    "4.0": "数据分析需求",
    "5.0": "技术开发需求",
    "nan": "UNK"
}
df_formated["reqType_cn"] = df_formated["reqType"].apply(lambda x: type_dict[str(x)])

In [31]:
df_formated["len"] = df_formated["doc_art_content_formated"].apply(lambda x: len(x))

In [32]:
df_formated

,docartname,doc_art_content,ssubmitdemandcode,reqName,reqType,status,doc_art_content_formated,status_cn,reqType_cn,len
0,解约协议文件导入,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n非联网产品接收渠道解约协议文件导入请求，逐条调用签约组件进行解约。\n界面原型...,需求已上线,数据分析需求,1040
1,导入结果查询,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n非联网产品接收渠道结果查询请求，返回渠道结果文件。\n界面原型*\n中间业务属...,需求已上线,数据分析需求,797
2,存量协议文件导入,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n非联网产品接收渠道存量协议导入请求，并逐条调用签约组件进行签约。\n界面原型*...,需求已上线,数据分析需求,1326
3,非联网签约,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n1 非联网产品接收渠道签约报文，调用签约组件进行签约。\n界...,需求已上线,数据分析需求,3606
4,?后管配置上线荣数SIP权益服务内容,"<div class=""odm-word"" data-docid=""690696198726...",YWXQ2021120438,权益管理系统接入荣数SIP需求报告,1.0,5,服务产品创建功能描述*\n1、 后管需要配置荣数SIP相关的权益服务内容。\n2、 对于服务...,需求已上线,产品研发需求,805
...,...,...,...,...,...,...,...,...,...,...
49005,贴现利率优惠券的领取和查询,"<div class=""odm-word"" data-docid=""709968605443...",CP0050323060525,票据宝贴现利率优惠券需求,1.0,"3,192",功能描述\n在企业互联pc端领取和查询优惠券。\n输入输出说明\n菜单新增位置：企业互联-票...,需求开发,产品研发需求,852
49006,拆出资金(行社之间)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192",功能描述*\n优化拆出资金（行社之间，包括本级）的SWIFT行社清算记账规则，取消核心手工记...,需求开发,产品研发需求,505
49007,拆出资金(境内境外)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192",功能描述*\n优化拆出资金（境内境外）的SWIFT行社清算记账规则，取消核心手工记账步骤，使...,需求开发,产品研发需求,857
49008,拆入资金(境内境外)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192",功能描述*\n优化拆入资金（境内境外）的SWIFT行社清算记账规则，取消核心手工记账步骤，使...,需求开发,产品研发需求,842


In [33]:
column_order = ["reqName", "ssubmitdemandcode", "reqType", "status_cn", "docartname"]
def reformat_text(item):
    text = f"需求类型：{item['reqType_cn']}" + "||| \n" \
        + f"需求编号：{item['ssubmitdemandcode']}" +  "||| \n" \
        + f"需求标题：{item['reqName']}" + "||| \n" \
        + f"流程状态：{item['status_cn']}" + "||| \n" \
        + f"条目名称：{item['docartname']}" + "||| \n" \
        + f"条目内容长度：{item['len']}" + "||| \n\n" \
        + f"条目内容：{item['doc_art_content_formated']}"
    return text

In [34]:
def reformat_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    df["doc_art_content_reformated"] = df.apply(lambda x: reformat_text(x), axis=1)
    return df

In [35]:
df_formated = reformat_dataframe(df=df_formated)
df_formated

,docartname,doc_art_content,ssubmitdemandcode,reqName,reqType,status,doc_art_content_formated,status_cn,reqType_cn,len,doc_art_content_reformated
0,解约协议文件导入,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n非联网产品接收渠道解约协议文件导入请求，逐条调用签约组件进行解约。\n界面原型...,需求已上线,数据分析需求,1040,需求类型：数据分析需求||| \n需求编号：YWXQ2022030149||| \n需求标题...
1,导入结果查询,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n非联网产品接收渠道结果查询请求，返回渠道结果文件。\n界面原型*\n中间业务属...,需求已上线,数据分析需求,797,需求类型：数据分析需求||| \n需求编号：YWXQ2022030149||| \n需求标题...
2,存量协议文件导入,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n非联网产品接收渠道存量协议导入请求，并逐条调用签约组件进行签约。\n界面原型*...,需求已上线,数据分析需求,1326,需求类型：数据分析需求||| \n需求编号：YWXQ2022030149||| \n需求标题...
3,非联网签约,"<div class=""odm-word"" data-docid=""690659647518...",YWXQ2022030149,金综风险监测预警新增规则开发需求,4.0,5,功能描述*\n1 非联网产品接收渠道签约报文，调用签约组件进行签约。\n界...,需求已上线,数据分析需求,3606,需求类型：数据分析需求||| \n需求编号：YWXQ2022030149||| \n需求标题...
4,?后管配置上线荣数SIP权益服务内容,"<div class=""odm-word"" data-docid=""690696198726...",YWXQ2021120438,权益管理系统接入荣数SIP需求报告,1.0,5,服务产品创建功能描述*\n1、 后管需要配置荣数SIP相关的权益服务内容。\n2、 对于服务...,需求已上线,产品研发需求,805,需求类型：产品研发需求||| \n需求编号：YWXQ2021120438||| \n需求标题...
...,...,...,...,...,...,...,...,...,...,...,...
49005,贴现利率优惠券的领取和查询,"<div class=""odm-word"" data-docid=""709968605443...",CP0050323060525,票据宝贴现利率优惠券需求,1.0,"3,192",功能描述\n在企业互联pc端领取和查询优惠券。\n输入输出说明\n菜单新增位置：企业互联-票...,需求开发,产品研发需求,852,需求类型：产品研发需求||| \n需求编号：CP0050323060525||| \n需求标...
49006,拆出资金(行社之间)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192",功能描述*\n优化拆出资金（行社之间，包括本级）的SWIFT行社清算记账规则，取消核心手工记...,需求开发,产品研发需求,505,需求类型：产品研发需求||| \n需求编号：JS0070123070068||| \n需求标...
49007,拆出资金(境内境外)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192",功能描述*\n优化拆出资金（境内境外）的SWIFT行社清算记账规则，取消核心手工记账步骤，使...,需求开发,产品研发需求,857,需求类型：产品研发需求||| \n需求编号：JS0070123070068||| \n需求标...
49008,拆入资金(境内境外)记账规则优化,"<div class=""odm-word"" data-docid=""708528144823...",JS0070123070068,外汇资金系统拆借业务SWIFT行社清算记账规则优化需求说明书,1.0,"3,192",功能描述*\n优化拆入资金（境内境外）的SWIFT行社清算记账规则，取消核心手工记账步骤，使...,需求开发,产品研发需求,842,需求类型：产品研发需求||| \n需求编号：JS0070123070068||| \n需求标...


### 1.3.数据去重

In [36]:
df_unique_formated = df_formated.drop_duplicates(subset="doc_art_content_formated")

In [37]:
df_unique_formated_shape = df_unique_formated.shape[0]
df_unique_formated_shape

32115

In [38]:
f"删除重复数据数量：{df_formated_shape - df_unique_formated_shape}"

'删除重复数据数量：16895'

### 1.3.删除错误数据

In [39]:
cond1 = df_unique_formated["docartname"].str.contains("功能清单")
cond2 = df_unique_formated["docartname"].str.contains("功能说明")
cond3 = df_unique_formated["docartname"].str.contains("功能描述")
cond4 = df_unique_formated["docartname"].str.contains("界面原型")
cond5 = df_unique_formated["docartname"].str.contains("业务要素")
cond6 = df_unique_formated["docartname"].str.contains("业务规则")
cond7 = df_unique_formated["docartname"].str.contains("输入输出")
cond8 = df_unique_formated["docartname"].str.contains("规则说明")
cond9 = df_unique_formated["docartname"].str.contains("其他说明")
cond10 = df_unique_formated["docartname"].str.contains("页面需求")
cond11 = df_unique_formated["docartname"].str.contains("权限说明")
cond = cond1 | cond2 | cond3 | cond4 | cond5 | cond6 | cond7 | cond8 | cond9 | cond10 | cond11
df_docartname = df_unique_formated[cond]
df_docartname.shape[0]

11219

In [40]:
df_unique_formated.drop(df_docartname.index, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [41]:
df_unique_formated.shape[0]

20896

### 1.4.关键词筛选

#### 1.4.1.页面

In [42]:
cond1 = df_unique_formated["doc_art_content_formated"].str.contains("修改字段")
cond2 = df_unique_formated["doc_art_content_formated"].str.contains("下拉枚举")
cond3 = df_unique_formated["doc_art_content_formated"].str.contains("UI界面")
cond = cond1 | cond2 | cond3
df1 = df_unique_formated[cond]
df1.shape[0]

81

#### 1.4.2.报表

In [43]:
cond1 = df_unique_formated["doc_art_content_formated"].str.contains("报表优化")
cond2 = df_unique_formated["doc_art_content_formated"].str.contains("报表需求")
cond3 = df_unique_formated["doc_art_content_formated"].str.contains("新增报表")
cond4 = df_unique_formated["doc_art_content_formated"].str.contains("报表描述")
cond5 = df_unique_formated["doc_art_content_formated"].str.contains("报表业务口径")
cond6 = df_unique_formated["doc_art_content_formated"].str.contains("报表查询")
cond = cond1 | cond2 | cond3 | cond4 | cond5 | cond6
df2 = df_unique_formated[cond]
df2.shape[0]

423

#### 1.4.3.指标

In [44]:
cond1 = df_unique_formated["doc_art_content_formated"].str.contains("指标需求")
cond2 = df_unique_formated["doc_art_content_formated"].str.contains("指标业务口径")
cond3 = df_unique_formated["doc_art_content_formated"].str.contains("指标优化")
cond = cond1 | cond2 | cond3
df3 = df_unique_formated[cond]
df3.shape[0]

400

#### 1.4.4.埋点

In [45]:
cond1 = df_unique_formated["doc_art_content_formated"].str.contains("埋点优化")
cond2 = df_unique_formated["doc_art_content_formated"].str.contains("数据埋点")
cond3 = df_unique_formated["doc_art_content_formated"].str.contains("点击位")
cond4 = df_unique_formated["doc_art_content_formated"].str.contains("采集数据")
cond5 = df_unique_formated["doc_art_content_formated"].str.contains("实际修订方式")
cond = cond1 | cond2 | cond3 | cond4 | cond5
df4 = df_unique_formated[cond]
df4.shape[0]

39

#### 1.4.8.接入

In [46]:
cond1 = df_unique_formated["doc_art_content_formated"].str.contains("接入")
cond2 = df_unique_formated["doc_art_content_formated"].str.contains("ODS")
cond3 = df_unique_formated["doc_art_content_formated"].str.contains("DW")
cond4 = df_unique_formated["doc_art_content_formated"].str.contains("SMY")
cond5 = df_unique_formated["doc_art_content_formated"].str.contains("Location")
cond6 = df_unique_formated["doc_art_content_formated"].str.contains("OLAP")
cond7 = df_unique_formated["doc_art_content_formated"].str.contains("SOPHON")
cond = cond1 | cond2 | cond3 | cond4 | cond5 | cond6 | cond7
df8 = df_unique_formated[cond]
df8.shape[0]

549

#### 1.4.9.下发

In [47]:
cond1 = df_unique_formated["doc_art_content_formated"].str.contains("下发")
cond2 = df_unique_formated["doc_art_content_formated"].str.contains("拆分方式")
cond = cond1 | cond2
df9 = df_unique_formated[cond]
df9.shape[0]

299

#### 1.4.10.开通推广

In [48]:
cond = df_unique_formated["doc_art_content_formated"].str.contains("开通")
df10 = df_unique_formated[cond]
df10.shape[0]

782

In [49]:
df_keyword_filter = pd.concat([df1, df2, df3, df4, df8, df9, df10])
df_keyword_filter_unique = df_keyword_filter.drop_duplicates(subset="doc_art_content_formated")
df_keyword_filter_unique.shape[0]

2474

#### 删除关键词过滤出的dataframe

In [50]:
df_unique_formated.shape[0]

20896

In [51]:
df_unique_formated.drop(df_keyword_filter_unique.index, inplace=True)

In [52]:
df_unique_formated.shape[0]

18422

### 1.5.需求编号筛选

In [53]:
con_fx = df_unique_formated["ssubmitdemandcode"].str.startswith("FX")
df_fx = df_unique_formated[con_fx]
df_fx.shape[0]

52

In [54]:
con_js = df_unique_formated["ssubmitdemandcode"].str.startswith("JS")
df_js = df_unique_formated[con_js]
df_js.shape[0]

389

In [55]:
df_unique_formated.shape[0]

18422

In [56]:
df_unique_formated.drop(df_fx.index, inplace=True)
df_unique_formated.drop(df_js.index, inplace=True)

In [57]:
df_unique_formated.shape[0]

17981

### 1.6.简单规则/复杂规则

In [58]:
con_js = df_unique_formated["ssubmitdemandcode"].str.startswith("YWXQ")
df_ywxq = df_unique_formated[~con_js]
df_ywxq.shape[0]

9230

In [59]:
df_ywxq_7085 = df_ywxq.head(7085)

### 1.6.数据保存

In [60]:
df_v1 = pd.concat([df_keyword_filter_unique, df_fx, df_js, df_ywxq_7085])
df_v1

,docartname,doc_art_content,ssubmitdemandcode,reqName,reqType,status,doc_art_content_formated,status_cn,reqType_cn,len,doc_art_content_reformated
59,新增需求文档分区功能,"<div class=""odm-word"" data-docid=""690946552449...",YWXQ2022030230,需求管理平台关于需求文档分区业务需求说明书,1.0,5,功能描述\n新增需求文档分区功能，实现ABC区的自定义配置。\n界面说明一、界面原型：界面一...,需求已上线,产品研发需求,1360,需求类型：产品研发需求||| \n需求编号：YWXQ2022030230||| \n需求标题...
70,报送-证监报送,"<div class=""odm-word"" data-docid=""690989319639...",YWXQ2022010138,财富代销系统（基金）证监数据报送需求报告,1.0,5,证监报送各张表增加报送字段功能描述*\n在证监报送菜单下的对应表格中，增加对应字段，用于行社...,需求已上线,产品研发需求,4076,需求类型：产品研发需求||| \n需求编号：YWXQ2022010138||| \n需求标题...
113,行社自建联盟功能新增,"<div class=""odm-word"" data-docid=""691056520187...",YWXQ2021070092,瑞丰银行_省卡劵核心融合需求,1.0,5,自发券创建功能描述*\n票券系统提供自发券创建接口；行社自建联盟创建自发券，经互联网前置调用...,需求已上线,产品研发需求,14128,需求类型：产品研发需求||| \n需求编号：YWXQ2021070092||| \n需求标题...
4680,外呼座席系统互联网融资催收字段优化,"<div class=""odm-word"" data-docid=""695048166486...",YWXQ2022070063,教育培训部-外呼服务持续优化（一）业务需求,1.0,5,功能描述*\n目前，互联网融资催收活动中，产品信息栏的“账单日”字段数据实际为还款日，需修改...,需求已上线,产品研发需求,81,需求类型：产品研发需求||| \n需求编号：YWXQ2022070063||| \n需求标题...
4876,XX模块（数据加工类模板）,"<div class=""odm-word"" data-docid=""694466459971...",YWXQ2022050110,丰收互联快捷方式设置优化详细业务需求说明书-v1.0 （小型类模板）,1.0,5,XX数据加工功能描述*\n描述需求所涉及的报表、指标、预警规则、绩效方案等的名称、目的、作用...,需求已上线,产品研发需求,699,需求类型：产品研发需求||| \n需求编号：YWXQ2022050110||| \n需求标题...
...,...,...,...,...,...,...,...,...,...,...,...
43317,产品画报-画报访问,"<div class=""odm-word"" data-docid=""708560070489...",CP0080023070484,理财有方系统小程序行社版画报转发,1.0,5,功能描述*\n接收到产品画报链接的微信好友可以点击链接跳转到理财有方小程序查看相应的产品详情...,需求已上线,产品研发需求,180,需求类型：产品研发需求||| \n需求编号：CP0080023070484||| \n需求标...
43318,视频画报-画报访问,"<div class=""odm-word"" data-docid=""708560070489...",CP0080023070484,理财有方系统小程序行社版画报转发,1.0,5,功能描述*\n接收到视频画报链接的微信好友可以点击链接跳转到理财有方小程序查看相应的产品详情...,需求已上线,产品研发需求,234,需求类型：产品研发需求||| \n需求编号：CP0080023070484||| \n需求标...
43319,零售客户关系管理--走访明细功能优化,"<div class=""odm-word"" data-docid=""708560273375...",CP0060823070073,【0703】零售客户关系管理-走访营销新增签到统计功能及走访明细功能优化需求说明书（小微类）,1.0,"3,193",功能描述*\n走访明细功能优化。\n输入输出\n界面/凭证/表单原型（样例）设计：\n无。\...,测试,产品研发需求,145,需求类型：产品研发需求||| \n需求编号：CP0060823070073||| \n需求标...
43320,零售客户关系管理--走访营销新增签到统计功能,"<div class=""odm-word"" data-docid=""708560273375...",CP0060823070073,【0703】零售客户关系管理-走访营销新增签到统计功能及走访明细功能优化需求说明书（小微类）,1.0,"3,193",功能描述*\n零售客户关系管理--走访营销新增签到统计功能。\n输入输出\n界面/凭证/表单...,测试,产品研发需求,461,需求类型：产品研发需求||| \n需求编号：CP0060823070073||| \n需求标...


In [61]:
df_v1["doc_art_content_reformated"].to_excel("./item_content_v1.0.0.xlsx", index=False, header=False, engine="xlsxwriter")

In [62]:
!jupyter nbconvert --to html convert_origin_to_label.ipynb

[NbConvertApp] Converting notebook convert_origin_to_label.ipynb to html
[NbConvertApp] Writing 726833 bytes to convert_origin_to_label.html


### 1.0.数据载入